In [1]:
# !pip install SpeechRecognition -qq

In [2]:
import speech_recognition as sr
from jiwer import compute_measures
import string
import numpy as np
r = sr.Recognizer()

In [3]:
from sk import get_files,predict,process_text,load_model
audios = get_files("/content/test-bahasa/",[".wav"],recurse=True)
audios

(#780) [PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-news-audio-1044.wav'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ182-000005.wav'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-streaming-kawasan pentadbiran dengan daripadanya.wav'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ178-000005.wav'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ219-000005.wav'),PosixPath('/content/test-bahasa/wattpad-audio-wattpad-1.wav'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ275-000005.wav'),PosixPath('/content/test-bahasa/iium-audio-iium-30.wav'),PosixPath('/content/test-bahasa/iium-audio-iium-8.wav'),PosixPath('/content/test-bahasa/iium-audio-iium-92.wav')...]

In [4]:
no_digit_text = [i.with_suffix(".txt").read_text() for i in audios]
with_digit_text = [i.with_suffix(".txt").read_text() for i in get_files("/content/test-bahasa-norm/",[".wav"],recurse=True)]

In [5]:
def google(audio):
    with sr.AudioFile(str(audio)) as source:
        a = r.record(source)

    text = r.recognize_google(a, language = 'ms')
    text = process_text(text)
    return text

In [6]:
%%time
from pqdm.threads import pqdm

g_texts = pqdm(audios,google,n_jobs=4)

SUBMITTING | : 100%|██████████| 780/780 [00:00<00:00, 47622.31it/s]
PROCESSING | : 100%|██████████| 780/780 [01:10<00:00, 11.03it/s]
COLLECTING | : 100%|██████████| 780/780 [00:00<00:00, 328469.59it/s]

CPU times: user 2.55 s, sys: 5.53 s, total: 8.08 s
Wall time: 1min 10s


In [7]:
g_d_wer = [compute_measures(i,j)["wer"] for i,j in zip(with_digit_text,g_texts)]
g_d_cer = [compute_measures([ii for ii in i],[jj for jj in j])["wer"] for i,j in zip(with_digit_text,g_texts)]
np.mean(g_d_wer),np.mean(g_d_cer) # with digit

(0.1526626638039393, 0.05278555178451872)

In [8]:
g_n_wer = [compute_measures(i,j)["wer"] for i,j in zip(no_digit_text,g_texts)]
g_n_cer = [compute_measures([ii for ii in i],[jj for jj in j])["wer"] for i,j in zip(no_digit_text,g_texts)]
np.mean(g_n_wer),np.mean(g_n_cer) # no digit

(0.1611180648863363, 0.060634040454885504)

In [9]:
# warm up
predict(audios[0],decoder="/content/out.trie.klm");

loading model
loading language model


100%|██████████| 1/1 [00:00<00:00, 11.39it/s]

Total input path: 1
Total audio found: 1
start prediction


In [10]:
%%time
texts,_,_,_ = predict(audios,decoder="/content/out.trie.klm")

loading model
loading language model
Total input path: 780


  0%|          | 2/780 [00:00<00:45, 16.96it/s]

Total audio found: 780
start prediction


100%|██████████| 780/780 [01:02<00:00, 12.56it/s]

CPU times: user 4min 4s, sys: 103 ms, total: 4min 4s
Wall time: 1min 2s


In [11]:
from words2num import normalize
n_texts = [normalize(i) for i in texts]
# with digit
sk_d_wer = [compute_measures(i,j)["wer"] for i,j in zip(with_digit_text,n_texts)]
sk_d_cer = [compute_measures([ii for ii in i],[jj for jj in j])["wer"] for i,j in zip(with_digit_text,n_texts)]
np.mean(sk_d_wer),np.mean(sk_d_cer) 

(0.1403347652637663, 0.03999910945695737)

In [12]:
# no digit
sk_n_wer = [compute_measures(i,j)["wer"] for i,j in zip(no_digit_text,texts)]
sk_n_cer = [compute_measures([ii for ii in i],[jj for jj in j])["wer"] for i,j in zip(no_digit_text,texts)]
np.mean(sk_n_wer),np.mean(sk_n_cer)

(0.13508409027804705, 0.036035160999825)

In [13]:
import pandas as pd
pd.set_option('max_colwidth', 400)
df = pd.DataFrame(zip(*[audios,no_digit_text,texts,n_texts,with_digit_text,g_texts]))
df.columns = ["path","no_digit_label","our","our_with_digit","with_digit_label","google"]
df.head()

,path,no_digit_label,our,our_with_digit,with_digit_label,google
0,/content/test-bahasa/-home-husein-speech-bahasa-news-audio-1044.wav,dan dokumen perjalanan bangladesh,dan dokumen perjalanan bangladesh,dan dokumen perjalanan bangladesh,dan dokumen perjalanan bangladesh,dan dokumen perjalanan bangladesh
1,/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ182-000005.wav,bagaimana anda akan menghentikan feminisme,bagaimana anda akan menghentikan faminism,bagaimana anda akan menghentikan faminism,bagaimana anda akan menghentikan feminisme,bagaimana anda akan menghentikan kami nizam
2,/content/test-bahasa/-home-husein-speech-bahasa-streaming-kawasan pentadbiran dengan daripadanya.wav,kawasan pentadbiran dengan daripadanya,kawasan pentadbiran dengan daripadanya,kawasan pentadbiran dengan daripadanya,kawasan pentadbiran dengan daripadanya,kawasan pentadbiran dengan daripadanya
3,/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ178-000005.wav,mengapa orang amerika putih takut kepada mexicans,mengapa orang amerika putih takut kepada maksikan,mengapa orang amerika putih takut kepada maksikan,mengapa orang amerika putih takut kepada mexicans,mengapa orang amerika putih takut kepada mexican
4,/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ219-000005.wav,daripada semua masalah dengan gereja mormon yang paling mengecewakan,daripada semua masalah dengan gereja mormon yang paling mengecewakan,daripada semua masalah dengan gereja mormon yang paling mengecewakan,daripada semua masalah dengan gereja mormon yang paling mengecewakan,daripada semua masalah dengan gereja mormon yang paling mengecewakan


In [14]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

In [15]:
df[df['with_digit_label'].apply(hasNumbers)].head(10)

,path,no_digit_label,our,our_with_digit,with_digit_label,google
63,/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ198-000005.wav,adakah ia baik untuk seorang wanita berumur dua belas tahun,adakah ia baik untuk seorang wanita berumur dua belas tahun,adakah ia baik untuk seorang wanita berumur 12 tahun,adakah ia baik untuk seorang wanita berumur 12 tahun,adakah ia baik untuk seorang wanita berumur 12 tahun
210,/content/test-bahasa/wattpad-audio-wattpad-47.wav,baru dua langkah dari gedung sekolah tiba tiba yasmin yang berada ditengah,baru dua langkah dari gedung sekolah tiba tiba yasmin yang berada di tengah,baru 2 langkah dari gedung sekolah tiba tiba yasmin yang berada di tengah,baru 2 langkah dari gedung sekolah tiba tiba yasmin yang berada ditengah,baru dua langkah dari gedung sekolah tiba tiba jasmine yang berada di tengah
229,/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ117-000005.wav,adakah buruk bahawa saya mencuri tujuh k barangan dari kedai runcit,adakah buruk bahawa saya mencuri tujuh ke barangan dari kedai runcit,adakah buruk bahawa saya mencuri 7 ke barangan dari kedai runcit,adakah buruk bahawa saya mencuri 7k barangan dari kedai runcit,adakah buruk bahawa saya mencuri 7k barangan dari kedai runcit
259,/content/test-bahasa/-home-husein-speech-bahasa-news-audio-1767.wav,studi ini juga mendapati sebanyak empat puluh tiga percent anak yang datang,studi ini juga mendapati sebanyak empat puluh tiga persen anak yang datang,studi ini juga mendapati sebanyak 43 persen anak yang datang,studi ini juga mendapati sebanyak 43 persen anak yang datang,study ini juga mendapati sebanyak 43 persen anak yang datang
266,/content/test-bahasa/-home-husein-speech-bahasa-news-audio-897.wav,phuket kochi dari rm tiga ratus sembilan puluh sembilan bali dari rm enam ratus dua puluh sembilan nanjing dari rm lima ratus lapan puluh sembilan,buket koci dari rm tiga ratus sembila puluh sembilan bali dari rm enam ratus dua puluh sembilan nanjing dari rm lima ratus lapan puluh sembilan,buket koci dari rm 300 sembila 9 bali dari rm 629 nanjing dari rm 589,phuket kochi dari rm359 bali dari rm629 nanjing dari rm589,phuket kochi dari rm359 bali dari rm600 29 nanjing dari rm589
296,/content/test-bahasa/-home-husein-speech-bahasa-news-audio-1529.wav,penelitian itu berdasarkan pada data lebih dari dua puluh tujuh ribu,penelitian itu berdasarkan pada data lebih dari dua puluh tujuh ribu,penelitian itu berdasarkan pada data lebih dari 27000,penelitian itu berdasarkan pada data lebih dari 27000,penelitian itu berdasarkan pada data lebih dari 27000
301,/content/test-bahasa/-home-husein-speech-bahasa-news-audio-394.wav,semenjak ianya ditubuhkan lebih dua puluh tahun yang lalu,semenjak ianya ditubuhkan lebih dua puluh tahun yang lalu,semenjak ianya ditubuhkan lebih 20 tahun yang lalu,semenjak ianya ditubuhkan lebih 20 tahun yang lalu,semenjak ianya ditubuhkan lebih 20 tahun yang lalu
313,/content/test-bahasa/wattpad-audio-wattpad-59.wav,ketika dia panik dia akan menangis selama kurang lebih lima belas minit menurut,ketika dia panik dia akan menangis selama kurang lebih lima belas minit menurut,ketika dia panik dia akan menangis selama kurang lebih 15 minit menurut,ketika dia panik dia akan menangis selama kurang lebih 15 minit menurut,ketika di planet dia akan menangis selama kurang lebih 15 menit menurut
322,/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ285-000005.wav,adakah wanita melakukannya dengan gaya anjing pada abad ke lapan belas,adakah wanita melakukannya dengan gaya anjing pada abad ke lapan belas,adakah wanita melakukannya dengan gaya anjing pada abad ke 18,adakah wanita melakukannya dengan gaya anjing pada abad ke 18,adakah wanita melakukannya dengan gaya anjing pada abad ke 18
345,/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ168-000005.wav,adakah ia ok kerana seorang lelaki berusia tiga puluh tujuh tahun,adakah ia okey kerana seorang lelaki berusia tiga puluh tujuh tahun,adakah ia okey kerana seorang 

# No digit dataset comparison

In [16]:
df = pd.DataFrame(zip(*[audios,no_digit_text,texts,g_texts,sk_n_cer,g_n_cer]))
df.columns = ["path","no_digit_label","our","google","our_no_digit_cer","google_no_digit_cer"]
df = df.sort_values("our_no_digit_cer",ascending=False)
df.head()

,path,no_digit_label,our,google,our_no_digit_cer,google_no_digit_cer
761,/content/test-bahasa/wattpad-audio-wattpad-646.wav,who are you,ayu,how are you,0.666667,0.222222
450,/content/test-bahasa/-home-husein-speech-bahasa-streaming-iaitu paus odontoceti paus.wav,iaitu paus odontoceti paus,iaitu pawesordantercetipawes,iaitu power order cathy paus,0.347826,0.521739
352,/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ118-000005.wav,boleh fuckers quora sila tutup fuck tentang iq,boleh pakar sekolah sila tutup fak tentang itu,boleh fuckers kura sila tutup port tentang itu,0.307692,0.205128
664,/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-ampe.wav,sebut perkataan ampe,sebuk pakatan ampa,sebut perkataan apa,0.277778,0.111111
613,/content/test-bahasa/-home-husein-speech-bahasa-tolong-sebut-jarum.wav,tolong sebut jarum,tolosebutjarol,tolong sebut jarum,0.250000,0.000000


our model not that good with english word because it not in the training set we use for community model

In [17]:
df = df.sort_values("google_no_digit_cer",ascending=False)
df.head()

,path,no_digit_label,our,google,our_no_digit_cer,google_no_digit_cer
442,/content/test-bahasa/wattpad-audio-wattpad-7.wav,bandung dua ribu tujuh belas,bandung dua ribu tujuh belas,bandung 2017,0.000000,0.708333
373,/content/test-bahasa/-home-husein-speech-bahasa-news-audio-1757.wav,dua puluh enam empat,dua puluh enam empat,26 empat,0.000000,0.705882
266,/content/test-bahasa/-home-husein-speech-bahasa-news-audio-897.wav,phuket kochi dari rm tiga ratus sembilan puluh sembilan bali dari rm enam ratus dua puluh sembilan nanjing dari rm lima ratus lapan puluh sembilan,buket koci dari rm tiga ratus sembila puluh sembilan bali dari rm enam ratus dua puluh sembilan nanjing dari rm lima ratus lapan puluh sembilan,phuket kochi dari rm359 bali dari rm600 29 nanjing dari rm589,0.032787,0.672131
541,/content/test-bahasa/wattpad-audio-wattpad-544.wav,libras pov,liberas pov,libras theory,0.111111,0.555556
690,/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-agihnya.wav,sebut perkataan agihnya,sebut perkataan agihnya,kata anaknya,0.000000,0.523810


most of the time google cer not good because it trying to predict digit for non digit dataset,that's why we provide separate dataset for fair comparison



# Digit dataset comparison

In [18]:
df = pd.DataFrame(zip(*[audios,with_digit_text,n_texts,g_texts,sk_d_cer,g_d_cer]))
df.columns = ["path","digit_label","our","google","our_digit_cer","google_digit_cer"]
df = df.sort_values("our_digit_cer",ascending=False)
df.head()

,path,digit_label,our,google,our_digit_cer,google_digit_cer
373,/content/test-bahasa/-home-husein-speech-bahasa-news-audio-1757.wav,26 empat,26 4,26 empat,0.714286,0.000000
761,/content/test-bahasa/wattpad-audio-wattpad-646.wav,who are you,ayu,how are you,0.666667,0.222222
168,/content/test-bahasa/-home-husein-speech-bahasa-streaming-sementara empat ion bergabung.wav,sementara empat ion bergabung,sementara 4 ion penggabuh,sementara empat aeon bergabung,0.384615,0.076923
450,/content/test-bahasa/-home-husein-speech-bahasa-streaming-iaitu paus odontoceti paus.wav,iaitu paus odontoceti paus,iaitu pawesordantercetipawes,iaitu power order cathy paus,0.347826,0.521739
551,/content/test-bahasa/-home-husein-speech-bahasa-streaming-sepanyol juta pada tahun.wav,sepanyol juta pada tahun,sepanyol 1000000 pada tahun,sepanyol juta pada tahun,0.333333,0.000000


- it seems that in written form even though it is a number if not more than 9 should not convert to digit
- for juta(million) should not convert to digit too I guess(need to fix my normalization rule)

let see what kind of error for google

In [19]:
df = df.sort_values("google_digit_cer",ascending=False)
df.head()

,path,digit_label,our,google,our_digit_cer,google_digit_cer
541,/content/test-bahasa/wattpad-audio-wattpad-544.wav,libras pov,liberas pov,libras theory,0.111111,0.555556
690,/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-agihnya.wav,sebut perkataan agihnya,sebut perkataan agihnya,kata anaknya,0.000000,0.523810
450,/content/test-bahasa/-home-husein-speech-bahasa-streaming-iaitu paus odontoceti paus.wav,iaitu paus odontoceti paus,iaitu pawesordantercetipawes,iaitu power order cathy paus,0.347826,0.521739
594,/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-ajal.wav,sebut perkataan ajal,sebut perkataan ajar,sebutkan saja,0.055556,0.444444
485,/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-adilkan.wav,sebut perkataan adilkan,sebut perkataan adilkan,perkataan ada dekat,0.000000,0.428571


- no comment except that google wrongly predict the text

# Disclaimer

- This benchmark only on Google Speech To Text that is free, the paid version might be better than this
- The performance might varies if we test with different test set and domain(TODO)